# Full MIDI Music Transformer Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# GPU check

In [ ]:
!nvidia-smi

# Setup environment

In [ ]:
!git clone https://github.com/asigalov61/tegridy-tools

In [ ]:
!pip install --upgrade pip
!pip install torch
!pip install einops
!pip install torch-summary
!pip install sklearn
!pip install tqdm
!pip install matplotlib

In [ ]:
# Load modules and make data dir

print('Loading modules...')

import os
import pickle
import random
import secrets
import tqdm

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

torch.set_float32_matmul_precision('high')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

%cd /content/tegridy-tools/tegridy-tools/

import TMIDIX

%cd /content/tegridy-tools/tegridy-tools/X-Transformer

from x_transformer import *

%cd /content/

if not os.path.exists('/content/INTS'):
    os.makedirs('/content/INTS')

import random
from pprint import pprint

print('Done')

# Load training data

In [ ]:
CHUNKS_LENGTH = 4096+4 # must be larger than model seq_len and must be divisible by 5
MIN_NUMBER_OF_CHUNK_EVENTS = 102 # min number of MIDI events per chunk
PAD_IDX = 1563 # model pad index

dataset_addr = "/content/INTS"

filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

filez.sort()

print('Loading training data... Please wait...')
print('=' * 70)

train_data = []

chunks_counter = 0
discarted_chunks_counter = 0

for f in tqdm.tqdm(filez[:20]):
  train_d = pickle.load(open(f, 'rb'))
  random.shuffle(train_d)
  for t in train_d:

    for i in range(0, len(t), CHUNKS_LENGTH):

      if 0 <= max(t[i:i+CHUNKS_LENGTH]) < PAD_IDX: # final data integrity check

        if len(t[i:i+CHUNKS_LENGTH]) == CHUNKS_LENGTH:
          train_data.append(t[i:i+CHUNKS_LENGTH])

        else:
          if len(t[i:i+CHUNKS_LENGTH]) > MIN_NUMBER_OF_CHUNK_EVENTS * 5:
            td = t[i:i+CHUNKS_LENGTH] + [PAD_IDX] * (CHUNKS_LENGTH-len(t[i:i+CHUNKS_LENGTH])) # padding with pad index
            train_data.append(td)
          else:
            discarted_chunks_counter += 1

        chunks_counter += 1

print('=' * 70)
print('Total number of imput chunks:', chunks_counter)
print('Total number of good chunks:', len(train_data))
print('Total number of discarted chunks:', discarted_chunks_counter, '/', round(100 * discarted_chunks_counter/chunks_counter, 3), '%')
print('All data is good:', len(max(train_data, key=len)) == len(min(train_data, key=len)))
print('=' * 70)
print('Final data randomization...')
random.shuffle(train_data)
print('Done!')
print('=' * 70)

In [ ]:
len(train_data)

In [ ]:
train_data[0][:15], train_data[0][-15:]

In [ ]:
len(train_data) / 16

# Setup model

In [ ]:
# Setup model

# constants

SEQ_LEN = 4096
BATCH_SIZE = 2
NUM_EPOCHS = 1
GRADIENT_ACCUMULATE_EVERY = 8

NUM_BATCHES = (len(train_data) // BATCH_SIZE // GRADIENT_ACCUMULATE_EVERY) * NUM_EPOCHS

LEARNING_RATE = 2e-4

VALIDATE_EVERY  = 100
SAVE_EVERY = 1000
GENERATE_EVERY  = 200
PRINT_STATS_EVERY = 50

GENERATE_LENGTH = 32

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

# instantiate the model

model = TransformerWrapper(
    num_tokens = 1564,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 1024, depth = 18, heads = 8)
)

model = AutoregressiveWrapper(model, ignore_index = 1563)

model = torch.nn.DataParallel(model)

model.cuda()

print('Done!')

summary(model)

# Dataloader

class MusicDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):

        # consequtive sampling

        full_seq = torch.Tensor(self.data[index][:self.seq_len+1]).long()

        return full_seq.cuda()

    def __len__(self):
        return (len(self.data) // BATCH_SIZE) * BATCH_SIZE

train_dataset = MusicDataset(train_data, SEQ_LEN)
val_dataset   = MusicDataset(train_data, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
train_dataset[0][:15]

In [ ]:
for t in train_loader:
  print(t)
  break

# Train

In [ ]:
# Train the model
import random


train_losses = []
val_losses = []

train_accs = []
val_accs = []

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='Training'):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss, acc = model(next(train_loader))
        loss.backward(torch.ones(loss.shape).cuda())

    if i % PRINT_STATS_EVERY == 0:
        print(f'Training loss: {loss.mean().item()}')
        print(f'Training acc: {acc.mean().item()}')

    train_losses.append(loss.mean().item())
    train_accs.append(acc.mean().item())

    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optim.step()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            val_loss, val_acc = model(next(val_loader))

            print(f'Validation loss: {val_loss.mean().item()}')
            print(f'Validation acc: {val_acc.mean().item()}')

            val_losses.append(val_loss.mean().item())
            val_accs.append(val_acc.mean().item())

            print('Plotting training loss graph...')

            tr_loss_list = train_losses
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')

            print('Plotting training acc graph...')

            tr_loss_list = train_accs
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')

            print('Plotting validation loss graph...')
            tr_loss_list = val_losses
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')

            print('Plotting validation acc graph...')
            tr_loss_list = val_accs
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]

        print(inp)

        sample = model.module.generate(inp[None, ...], GENERATE_LENGTH)

        print(sample)

    if i % SAVE_EVERY == 0:

        print('Saving model progress. Please wait...')
        print('model_checkpoint_' + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth')

        fname = '/content/drive/MyDrive/OUT/model_checkpoint_' + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth'

        torch.save(model.state_dict(), fname)

        data = [train_losses, train_accs, val_losses, val_accs]

        TMIDIX.Tegridy_Any_Pickle_File_Writer(data, '/content/drive/MyDrive/OUT/losses_accs')

        print('Done!')

# Final Save

In [ ]:
print('Saving model progress. Please wait...')
print('model_checkpoint_' + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth')

fname = '/content/model_checkpoint_' + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth'

torch.save(model.state_dict(), fname)

print('Done!')

In [ ]:
# Save training loss graph

plt.plot([i for i in range(len(train_losses))] ,train_losses, 'b')
plt.savefig('/content/training_loss_graph.png')
plt.close()
print('Done!')

# Save training acc graph

plt.plot([i for i in range(len(train_accs))] ,train_accs, 'b')
plt.savefig('/content/training_acc_graph.png')
plt.close()
print('Done!')

# Save validation loss graph

plt.plot([i for i in range(len(val_losses))] ,val_losses, 'b')
plt.savefig('/content/validation_loss_graph.png')
plt.close()
print('Done!')

# Save validation acc graph

plt.plot([i for i in range(len(val_accs))] ,val_accs, 'b')
plt.savefig('/content/validation_acc_graph.png')
plt.close()
print('Done!')

In [ ]:
data = [train_losses, train_accs, val_losses, val_accs]

TMIDIX.Tegridy_Any_Pickle_File_Writer(data, '/content/losses_accuracies')

# Eval

In [ ]:
#@title Generate

model.eval()

inp = [random.choice(val_dataset)[:500].tolist()]

inp = torch.LongTensor(inp).cuda()

print(inp)

sample = model.module.generate(inp, 500, temperature=0.8, return_prime=True)

print(sample)

In [ ]:
pprint(sample[0].tolist(), compact=True)

In [ ]:
#@title Test INTs

data = sample[0].tolist()

print('Sample INTs', data[:15])

if len(data) != 0:

    song = data
    song_f = []
    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    son = []
    song1 = []
    for i in range(0, len(song), 5): # creating penta seqs...
        song1.append(song[i:i+5])

    patch_list = [0] * 16
    patch_list[9] = 128

    channels_list = [0] * 16
    channels_list[9] = 1

    for s in song1: # decoding...

        # 1553 - pad token

        # 1554 - patch change token
        # 1555 - control change token
        # 1556 - key after touch token
        # 1557 - channel after touch token
        # 1558 - pitch wheel change token
        # 1559 - counters seq token

        # 1560 - outro token
        # 1561 - end token
        # 1562 - start token

        if s[0] < 256: # Note

            patch = s[0]
            time += (s[1]-256) * 16
            dur = (s[2]-256-128) * 32
            pitch = (s[3]-256-128-128) % 128
            vel = (s[4]-256-128-128-256)

            if patch in patch_list:
                channel = patch_list.index(patch)
                channels_list[channel] = 1

            else:
                if 0 in channels_list:
                  channel = channels_list.index(0)
                  channels_list[channel] = 1
                  song_f.append(['patch_change', time, channel, patch])

                else:
                  channel = 15
                  channels_list[channel] = 1
                  song_f.append(['patch_change', time, channel, patch])

            song_f.append(['note', time, dur, channel, pitch, vel])

        if s[0] == 1554: # patch change

            time += (s[1]-256) * 16
            channel = (s[2]-(256+128+128+256+128))
            patch = s[3]

            if channel != 9:
                patch_list[channel] = patch
            else:
                patch_list[channel] = patch + 128

            song_f.append(['patch_change', time, channel, patch])

        if s[0] == 1555: # control change

            time += (s[1]-256) * 16
            patch = s[2]
            controller = (s[3]-(256+128+128+256+128+16))
            controller_value = (s[4]-(256+128+128+256+128+16+128))

            try:
                channel = patch_list.index(patch)
            except:
                channel = 15

            song_f.append(['control_change', time, channel, controller, controller_value])

        if s[0] == 1556: # key after touch

            time += (s[1]-256) * 16
            patch = s[2]
            pitch = (s[3]-256-128-128) % 128
            vel = (s[4]-256-128-128-256)

            try:
                channel = patch_list.index(patch)
            except:
                channel = 15

            song_f.append(['key_after_touch', time, channel, pitch, vel])

        if s[0] == 1557: # channel after touch

            time += (s[1]-256) * 16
            patch = s[2]
            vel = (s[3]-256-128-128-256)

            try:
                channel = patch_list.index(patch)
            except:
                channel = 15

            song_f.append(['channel_after_touch', time, channel, vel])

        if s[0] == 1558: # pitch wheel change

            time += (s[1]-256) * 16
            patch = s[2]
            pitch_wheel = (s[3]-(256+128+128+256+128+16+128)) * 128

            try:
                channel = patch_list.index(patch)
            except:
                channel = 15

            song_f.append(['pitch_wheel_change', time, channel, pitch_wheel])

detailed_stats = TMIDIX.Tegridy_SONG_to_Full_MIDI_Converter(song_f,
                                                    output_signature = 'FMT',
                                                    output_file_name = '/content/LAMD-Music-Composition',
                                                    track_name='Project Los Angeles'
                                                    )

# Congrats! You did it! :)